<a href="https://colab.research.google.com/github/h0806449f/PyTorch/blob/main/LightningAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **== Setups ==**

## import

In [ ]:
!pip install torchinfo

In [ ]:
!pip install gitpython

In [ ]:
!pip install lightning

In [4]:
import os
import pandas as pd
from git import Repo
from PIL import Image
from collections import Counter

In [5]:
import torch
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import torchvision
from torchvision import datasets, transforms

from torchinfo import summary

from lightning import LightningModule
from lightning import Trainer

## functions

In [6]:
def train(model, dataloader, loss_fn, optimizer, device):
    model.to(device)
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X_train, y_train) in enumerate(dataloader):
        X_train, y_train = X_train.to(device), y_train.to(device)

        train_pred = model(X_train)

        loss = loss_fn(train_pred, y_train)
        train_loss = train_loss + loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_pred_label = torch.argmax(torch.softmax(train_pred, dim = 1), dim = 1)
        train_acc = train_acc + (train_pred_label == y_train).sum().item() / len(train_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

In [7]:
def test(model, dataloader, loss_fn, device):
    model.to(device)
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X_test, y_test) in enumerate(dataloader):
            X_test, y_test = X_test.to(device), y_test.to(device)

            test_pred = model(X_test)

            loss = loss_fn(test_pred, y_test)
            test_loss = test_loss + loss.item()

            test_pred_label = torch.argmax(torch.softmax(test_pred, dim = 1), dim = 1)
            test_acc = test_acc + (test_pred_label == y_test).sum().item() / len(test_pred)

        test_loss = test_loss / len(dataloader)
        test_acc = test_acc / len(dataloader)

        return test_loss, test_acc

In [8]:
def val(model, dataloader, loss_fn, device):
    model.to(device)
    model.eval()

    val_loss, val_acc = 0, 0

    with torch.inference_mode():
        for batch, (X_val, y_val) in enumerate(dataloader):
            X_val, y_val = X_val.to(device), y_val.to(device)

            val_pred = model(X_val)

            loss = loss_fn(val_pred, y_val)
            val_loss = val_loss + loss.item()

            val_pred_label = torch.argmax(torch.softmax(val_pred, dim = 1), dim = 1)
            val_acc = val_acc + (val_pred_label == y_val).sum().item() / len(val_pred)

        val_loss = val_loss / len(dataloader)
        val_acc = val_acc / len(dataloader)

        return val_loss, val_acc

In [70]:
def train_val_loop(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs, device):
    results = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}

    for epoch in range(epochs):
        train_loss, train_acc = train(model = model,
                                      dataloader = train_dataloader,
                                      loss_fn = loss_fn,
                                      optimizer = optimizer,
                                      device = device)

        val_loss, val_acc = val(model = model,
                                dataloader = val_dataloader,
                                loss_fn = loss_fn,
                                device = device)

        print(f"Epoch: {epoch+1}\n"
              f"Train loss: {train_loss:.4f} | Train acc: {(train_acc*100):.2f}%\n"
              f"Val loss: {val_loss:.4f} | Val acc: {(val_acc*100):.2f}%"
              )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)

    return results

# **== MNIST ==**

## Process

transform


In [10]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = 0.5, std = 0.5)
])

dataset

In [ ]:
train_dataset = datasets.MNIST(
    root = "./data/mnist",
    train = True,
    transform = transform,
    download = True)

test_dataset = datasets.MNIST(
    root = "./data/mnist",
    train = False,
    transform = transform,
    download = True)

split dataset

In [12]:
train_dataset, val_dataset = random_split(train_dataset,
                                          lengths=[50000, 10000])

In [13]:
len(train_dataset), len(val_dataset), len(test_dataset)

(50000, 10000, 10000)

dataloader

In [14]:
train_dataloader = DataLoader(train_dataset,
                              batch_size = 32,
                              shuffle = True,
                              drop_last = True)

val_dataloader = DataLoader(val_dataset,
                            batch_size = 32,
                            shuffle = False,
                            drop_last = True)

test_dataloader = DataLoader(test_dataset,
                             batch_size = 32,
                             shuffle = False,
                             drop_last = True)

check dataloader distribution

In [15]:
# train_counter = Counter()

# for imgs, labels in train_dataloader:
#     train_counter.update(labels.tolist())

# print("Training label distribution:")
# print(sorted(train_counter.items()))

In [16]:
# print(f"{imgs.shape} -> batch_szie, channel, height, width")

In [17]:
class MNIST_model_1(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()

        self.flat = nn.Flatten()

        self.layers = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p = 0.2),
            nn.Linear(128, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(p = 0.2),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        x = self.flat(x)
        x = self.layers(x)

        return x

MNIST_model_1 = MNIST_model_1(num_features = 1*28*28, num_classes = 10)

## Training and results

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(MNIST_model_1.parameters(),
                             lr = 0.001)

MNIST_model_1_results = train_test_loop(model = MNIST_model_1,
                                        train_dataloader = train_dataloader,
                                        test_dataloader = val_dataloader,
                                        loss_fn = loss_fn,
                                        optimizer = optimizer,
                                        epochs = 10,
                                        device = device)

Epoch: 1
Train loss: 0.4407 | Train acc: 88.78%
Val loss: 0.1466 | Val acc: 95.54%
Epoch: 2
Train loss: 0.2284 | Train acc: 93.19%
Val loss: 0.1040 | Val acc: 96.64%
Epoch: 3
Train loss: 0.1847 | Train acc: 94.43%
Val loss: 0.0984 | Val acc: 96.99%
Epoch: 4
Train loss: 0.1661 | Train acc: 94.90%
Val loss: 0.0889 | Val acc: 97.04%
Epoch: 5
Train loss: 0.1465 | Train acc: 95.56%
Val loss: 0.0813 | Val acc: 97.37%
Epoch: 6
Train loss: 0.1430 | Train acc: 95.64%
Val loss: 0.0808 | Val acc: 97.45%
Epoch: 7
Train loss: 0.1313 | Train acc: 95.98%
Val loss: 0.0788 | Val acc: 97.53%
Epoch: 8
Train loss: 0.1236 | Train acc: 96.20%
Val loss: 0.0750 | Val acc: 97.62%
Epoch: 9
Train loss: 0.1198 | Train acc: 96.29%
Val loss: 0.0730 | Val acc: 97.63%
Epoch: 10
Train loss: 0.1143 | Train acc: 96.45%
Val loss: 0.0723 | Val acc: 97.68%


In [19]:
test_loss_1, test_acc_1 = test(model = MNIST_model_1,
                           dataloader = test_dataloader,
                           loss_fn = loss_fn,
                           device = device)
print(f"Test loss: {test_loss_1:.4f} | Test acc: {(test_acc_1*100):.2f}%")

Test loss: 0.0697 | Test acc: 97.87%


# **== Custom dataset, dataloader ==**

function

In [20]:
# class CustomDataset(Dataset):
#     # Set up attributes
#     def __init__(self, csv_path, img_dir, transform = None):
#         df = pd.read_csv(csv_path)
#         self.img_dir = img_dir
#         self.transform = transform

#         self.img_names = df["filepath"]
#         self.labels = df["label"]

#     # Define how to get single record
#     def __getitem__(self, index):
#         img = Image.open(os.path.join(self.img_dir, self.img_names[index]))

#         if self.transform is not None:
#             img = self.transform(img)

#         label = self.labels[index]

#         return img, label

#     # Return the length of dataset
#     def __len__(self):
#         return self.labels.shape[0]

raw dara -> pandas df

In [21]:
# # Download
# if not os.path.exists("mnist-pngs"):
#     Repo.clone_from("https://github.com/rasbt/mnist-pngs", "mnist-pngs")

# # Read csv
# df_train = pd.read_csv("mnist-pngs/train.csv")
# df_test = pd.read_csv("mnist-pngs/test.csv")

# # 順序打亂 (為了使val dataset 也包含多樣資料)
# df_train = df_train.sample(frac = 1, random_state = 123)

# # Split train & val dataset
# split_index = round(df_train.shape[0] * 0.9)

# df_new_train = df_train.iloc[:split_index]
# df_new_val = df_train.iloc[split_index:]

# df_new_train.to_csv("mnist-pngs/new_train.csv", index=None)
# df_new_val.to_csv("mnist-pngs/new_val.csv", index=None)

transform

In [22]:
# data_transforms = {
#     "train": transforms.Compose([
#         transforms.Resize(32),
#         transforms.RandomCrop((28, 28)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean = 0.5, std = 0.5)
#     ]),
#     "test": transforms.Compose([
#         transforms.Resize(32),
#         transforms.CenterCrop((28, 28)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean = 0.5, std = 0.5)
#     ])
# }

dataset -> dataloader

In [23]:
# train_dataset = CustomDataset(
#     csv_path = "mnist-pngs/new_train.csv",
#     img_dir = "mnist-pngs/",
#     transform = data_transforms["train"])

# train_dataloader = DataLoader(
#     dataset = train_dataset,
#     batch_size = 32,
#     shuffle = True)

# **== CNN -> regression dataset ==**

In [24]:
# Normal CNN
"""
    input
    hidden layers
    softmax
    argmax
    output layers: number of class names
"""

# regression
"""
    input
    hidden layers
    output layers: 1
    loss_fn -> Meas Squared Error -> nn.MSELoss()
"""

'\n    input\n    hidden layers\n    output layers: 1\n    loss_fn -> Meas Squared Error -> nn.MSELoss()\n'

# **== CNN -> CIFAR10 ==**

## Process

transforms

In [57]:
weights = torchvision.models.ResNet18_Weights.DEFAULT

transform = weights.transforms()

dataset

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root = "./data",
                                             train = True,
                                             download = True,
                                             transform = transform)

val_dataset = torchvision.datasets.CIFAR10(root = "./data",
                                             train = False,
                                             download = True,
                                             transform = transform)

test_dataset = torchvision.datasets.CIFAR10(root = "./data",
                                            train = False,
                                            download = True,
                                            transform = transform)

dataloader

In [76]:
BATCH_SIZE = 8

train_dataloader = DataLoader(dataset = train_dataset,
                              batch_size = BATCH_SIZE,
                              shuffle = True,
                              drop_last = True)

val_dataloader = DataLoader(dataset = val_dataset,
                            batch_size = BATCH_SIZE,
                            shuffle = False,
                            drop_last = False)

test_dataloader = DataLoader(dataset = test_dataset,
                             batch_size = BATCH_SIZE,
                             shuffle = False,
                             drop_last = False)

model

In [62]:
weights = torchvision.models.ResNet18_Weights.DEFAULT

CIFAR10_model1 = torchvision.models.resnet18(weights = weights)

In [65]:
for param in CIFAR10_model1.parameters():
    param.requires_grad = False

In [68]:
CIFAR10_model1_classifier = nn.Sequential(
    nn.ReLU(),
    nn.Dropout(p = 0.3),
    nn.Linear(1000, 128),
    nn.ReLU(),
    nn.Dropout(p = 0.2),
    nn.Linear(128, 10)

)

CIFAR10_model1 = nn.Sequential(
    CIFAR10_model1,
    CIFAR10_model1_classifier
)

In [69]:
summary(CIFAR10_model1,
        input_size = (1, 3, 32, 32),
        col_names = ["output_size", "num_params", "trainable"],
        col_width = 17)

Layer (type:depth-idx)                        Output Shape      Param #           Trainable
Sequential                                    [1, 10]           --                Partial
├─ResNet: 1-1                                 [1, 1000]         --                False
│    └─Conv2d: 2-1                            [1, 64, 16, 16]   (9,408)           False
│    └─BatchNorm2d: 2-2                       [1, 64, 16, 16]   (128)             False
│    └─ReLU: 2-3                              [1, 64, 16, 16]   --                --
│    └─MaxPool2d: 2-4                         [1, 64, 8, 8]     --                --
│    └─Sequential: 2-5                        [1, 64, 8, 8]     --                False
│    │    └─BasicBlock: 3-1                   [1, 64, 8, 8]     (73,984)          False
│    │    └─BasicBlock: 3-2                   [1, 64, 8, 8]     (73,984)          False
│    └─Sequential: 2-6                        [1, 128, 4, 4]    --                False
│    │    └─BasicBlock: 3-3     

## Training and results
took 40 mins

In [73]:
device = "cuda" if torch.cuda.is_available() else "cpu"

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CIFAR10_model1.parameters(),
                             lr = 0.001)

In [74]:
results_CIFAR10_model1 = train_val_loop(model = CIFAR10_model1,
                                        train_dataloader = train_dataloader,
                                        test_dataloader = val_dataloader,
                                        loss_fn = loss_fn,
                                        optimizer = optimizer,
                                        epochs = 10,
                                        device = device)

Epoch: 1
Train loss: 1.1980 | Train acc: 59.01%
Val loss: 0.8663 | Val acc: 69.91%
Epoch: 2
Train loss: 1.1302 | Train acc: 61.27%
Val loss: 0.8134 | Val acc: 72.18%
Epoch: 3
Train loss: 1.1144 | Train acc: 62.28%
Val loss: 0.7918 | Val acc: 73.07%
Epoch: 4
Train loss: 1.1039 | Train acc: 62.43%
Val loss: 0.8165 | Val acc: 72.14%
Epoch: 5
Train loss: 1.0965 | Train acc: 63.17%
Val loss: 0.8029 | Val acc: 72.89%
Epoch: 6
Train loss: 1.0949 | Train acc: 63.39%
Val loss: 0.7946 | Val acc: 72.79%
Epoch: 7
Train loss: 1.0892 | Train acc: 63.37%
Val loss: 0.8187 | Val acc: 72.32%
Epoch: 8
Train loss: 1.0890 | Train acc: 63.62%
Val loss: 0.8098 | Val acc: 72.02%
Epoch: 9
Train loss: 1.0890 | Train acc: 63.35%
Val loss: 0.8044 | Val acc: 73.02%
Epoch: 10
Train loss: 1.0880 | Train acc: 63.31%
Val loss: 0.7658 | Val acc: 73.95%


In [77]:
test_loss_2, test_acc_2 = test(model = CIFAR10_model1,
                               dataloader = test_dataloader,
                               loss_fn = loss_fn,
                               device = device)

print(f"Test loss: {test_loss_2:.4f} | Test acc: {(test_acc_2*100):.2f}%")

Test loss: 0.7658 | Test acc: 73.95%


## Save, load, predict

save model

In [86]:
torch.save(CIFAR10_model1, "./data/mode_1forCIFAR10.pt")

load model

In [ ]:
model = torch.load("/content/data/mode_1forCIFAR10.pt")

predict

In [104]:

model.eval()

image, label = next(iter(test_dataset))
image = image.unsqueeze(dim = 0).to(device)

logits = model(image)
pred_probality = torch.softmax(logits, dim = 1)
pred_label = torch.argmax(pred_probality, dim = 1).item()

print(f"Predict label: {pred_label} | Actual label: {label}")

Predict label: 3 | Actual label: 3


# **== ViT -> image ==**

# **== temp ==**

# **==temp==**

# **==temp==**